In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_adult = pd.read_csv('adult.data')
df_letter = pd.read_csv('letter-recognition.data')

# **Adults**

In [3]:
df_adult.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [4]:
df_adult.shape

(32560, 15)

In [5]:
df_adult.columns = df_adult.columns.str.replace(' ', '')

In [6]:
df_adult.columns

Index(['39', 'State-gov', '77516', 'Bachelors', '13', 'Never-married',
       'Adm-clerical', 'Not-in-family', 'White', 'Male', '2174', '0', '40',
       'United-States', '<=50K'],
      dtype='object')

In [7]:
df_adult['<=50K'].unique()

array([' <=50K', ' >50K'], dtype=object)

### Selecting random datapoints

In [15]:
# test dataset
p = pd.DataFrame(data={'a':[1,55,456,223,789,43],'b':[90,24,4325,4675,2314,43],'c':[123,324,12334,543,345,129],'d':[3214,1234,325,453,213,567]})
p

,a,b,c,d
0,1,90,123,3214
1,55,24,324,1234
2,456,4325,12334,325
3,223,4675,543,453
4,789,2314,345,213
5,43,43,129,567


In [17]:
pd.get_dummies(df_adult, columns=['State-gov', 'Bachelors', 'Adm-clerical','Never-married']).head()

,39,State-gov,77516,Bachelors,13,Adm-clerical,Not-in-family,White,Male,2174,...,40,United-States,<=50K,Never-married_ Divorced,Never-married_ Married-AF-spouse,Never-married_ Married-civ-spouse,Never-married_ Married-spouse-absent,Never-married_ Never-married,Never-married_ Separated,Never-married_ Widowed
0,50,Self-emp-not-inc,83311,Bachelors,13,Exec-managerial,Husband,White,Male,0,...,13,United-States,<=50K,0,0,1,0,0,0,0
1,38,Private,215646,HS-grad,9,Handlers-cleaners,Not-in-family,White,Male,0,...,40,United-States,<=50K,1,0,0,0,0,0,0
2,53,Private,234721,11th,7,Handlers-cleaners,Husband,Black,Male,0,...,40,United-States,<=50K,0,0,1,0,0,0,0
3,28,Private,338409,Bachelors,13,Prof-specialty,Wife,Black,Female,0,...,40,Cuba,<=50K,0,0,1,0,0,0,0
4,37,Private,284582,Masters,14,Exec-managerial,Wife,White,Female,0,...,40,United-States,<=50K,0,0,1,0,0,0,0


In [35]:
df_adult.dtypes

39                int64
State-gov        object
77516             int64
Bachelors        object
13                int64
Never-married    object
Adm-clerical     object
Not-in-family    object
White            object
Male             object
2174              int64
0                 int64
40                int64
United-States    object
<=50K            object
dtype: object

In [48]:
categorical = [i for i in df_adult.dtypes.index if df_adult.dtypes[i]=='object']

In [50]:
pd.get_dummies(df_adult, columns=categorical).head()

,39,77516,13,2174,0,40,State-gov_ ?,State-gov_ Federal-gov,State-gov_ Local-gov,State-gov_ Never-worked,...,United-States_ Scotland,United-States_ South,United-States_ Taiwan,United-States_ Thailand,United-States_ Trinadad&Tobago,United-States_ United-States,United-States_ Vietnam,United-States_ Yugoslavia,<=50K_ <=50K,<=50K_ >50K
0,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,37,284582,14,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
